# Importing Multilingual Data

In [2]:
import pandas as pd
import csv
import creds

df = pd.read_csv("/data/shared/reddit_data_multilingual/data_full/balanced/data-en/train.csv", index_col=0, quoting=csv.QUOTE_NONE, # Multilingual Dataset
    sep=',',
)

islam_df = df[df['subreddit'] == 'islam'] # Looking at only islam
islam_df = islam_df[islam_df['label'] == 1] # Looking for only removed comments
islam_df = islam_df.reset_index(drop=False)
islam_df.head()

,index,text,label,id,subreddit
0,320,actually {{organization}} recently changed its...,1,idntc8o,islam
1,495,ignorant cuz i refuse to accept the cult or ac...,1,ik5hdol,islam
2,649,there is no afterlife when you die its just n...,1,id1j3qz,islam
3,954,islam aligns itself with pedophilia and killin...,1,ibe7j8g,islam
4,1108,instead of cleaning their dirty streets ....,1,io106a1,islam


There is about 10000 entries for r/islam

Rules for r/islam:

1. Be respectful at all times and conduct yourself in a civil manner.
2. No personal information or illegal content.
3. No harassment or witch-hunting.
4. Do not derail posts.
5. Do not proselytize.
6. Do not engage in behaviour that encourages vote manipulation or brigading.
7. Do not post any NSFW content without prior approval by a moderator.
8. Do not engage in sectarianism.
9. Do not give or imply any rulings or religious edicts.
10. No advertising, self-promotion, fundraising, or data collection.
11. No FAQs or posts addressed in the wiki.
12. All posts must meet the submission guidelines.

Rules for r/Judaism
1. Don't be a jerk
2. Don't proselytize.
3. No antisemitism.
4. Don't make clickbait.
5. Don't fundraise/survey without prior permission.
6. If it's worth reporting, report and don't respond
7. Reports are most useful when you give details
8. No Politics
9. No reposts

# Using PRAW to Scrape Post Text

We need to get scrape the text from submission to use as context for our prompt

Create Reddit Instance from PRAW API

In [3]:
import praw

reddit = praw.Reddit(
    client_id=creds.client_id,
    client_secret=creds.client_secret,
    password=creds.password,
    user_agent=creds.user_agent,
    username=creds.username,
)

#### findSubmission(df):

df needs a column 'id' for this to work 

Given Comments, finds each comment's respective submission

In [4]:
def findSubmission(df):
    comment_ids = df['id'].values.tolist()
    comment_ids = ['t1_' + id for id in comment_ids]
    submissions = []

    batch_size = 100  # PRAW can read up to 100 comment/submission ids at a time so batch

    for i in range(0, len(comment_ids), batch_size):    
        batch_comment_ids = comment_ids[i:i + batch_size]
        comments = reddit.info(fullnames=batch_comment_ids)
        for comment in comments:
            submissions.append(str(comment.submission))
            
    return submissions

#### findSubmissionText(df)

df needs a column 'submission_id'

given submission id, find text

In [5]:
def findSubmissionText(df):
    submission_ids = df['submission_id'].values.tolist()
    submission_ids = ['t3_' + id for id in submission_ids]
    submission_text = []

    batch_size = 100  

    for i in range(0, len(submission_ids), batch_size):
        batch_comment_ids = submission_ids[i:i + batch_size]
        submissions = reddit.info(fullnames=batch_comment_ids)
        for submission in submissions:
            submission_text.append(str(submission.title) + " - " + str(submission.selftext))
            
    return submission_text

#### findSubmissionCommentNum(df)

given submission id, find amount of comments

In [6]:
def findSubmissionCommentNum(df):
    submission_ids = df['submission_id'].values.tolist()
    submission_ids = ['t3_' + id for id in submission_ids]
    submission_text = []

    batch_size = 100  

    for i in range(0, len(submission_ids), batch_size):
        batch_comment_ids = submission_ids[i:i + batch_size]
        submissions = reddit.info(fullnames=batch_comment_ids)
        for submission in submissions:
            submission_text.append(submission.num_comments)
            
    return submission_text


# Creating a testdf with subsets of islam_df

In [7]:
test_df = islam_df[:1000]
test_df['submission_id'] = findSubmission(test_df)
test_df['submission_text'] = findSubmissionText(test_df)
test_df['comment_num'] = findSubmissionCommentNum(test_df)

/tmp/ipykernel_13970/3364561719.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['submission_id'] = findSubmission(test_df)
/tmp/ipykernel_13970/3364561719.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['submission_text'] = findSubmissionText(test_df)
/tmp/ipykernel_13970/3364561719.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

### Cleaning the data

We only want comment num < 20 (about 85 in first 1000 comments in dataset)

In [8]:
test_df = test_df[~test_df['submission_text'].str.contains(r'\[removed\]')]
test_df = test_df[test_df['comment_num'] < 20]
test_df = test_df.reset_index(drop=True)
test_df.head()

,index,text,label,id,subreddit,submission_id,submission_text,comment_num
0,1980,if you missed them intentionally than you need...,1,iiejfel,islam,wctaej,Missed Zuhr and Ishaa prayers intentionally. -...,4
1,2184,and aisha was a virgin until the end of her life,1,i3uzwgw,islam,tyeg2p,So for some reason this islamaphobic argument ...,12
2,4094,it's not a satanic pentagram you uncultured sw...,1,i81xj5x,islam,ulzfbo,Moroccan Ministry of Awqaf and Islamic Affairs...,3
3,4826,haram haram. just read the quran and allah wil...,1,i194hqb,islam,thnice,Is Medical Assisted Suicide Halal? - Say you h...,4
4,4861,is a 100km journey by car difficult enough to ...,1,i4js4ad,islam,u2hg74,"Can I break my fast? - Hey, I am going to anot...",3


# Creating Another TestDf using Judaism

In [9]:
judaism_df = df[df['subreddit'] == 'judaism']
judaism_df = judaism_df[judaism_df['label'] == 1]
judaism_df = judaism_df.reset_index(drop=False)

test2_df = judaism_df[:1000]
test2_df['submission_id'] = findSubmission(test2_df)
test2_df['submission_text'] = findSubmissionText(test2_df)
test2_df['comment_num'] = findSubmissionCommentNum(test2_df)

test2_df = test2_df[~test2_df['submission_text'].str.contains(r'\[removed\]')]
test2_df = test2_df[test2_df['comment_num'] < 20]
test2_df = test2_df.reset_index(drop=True)
test2_df.head()

/tmp/ipykernel_13970/4182242450.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2_df['submission_id'] = findSubmission(test2_df)
/tmp/ipykernel_13970/4182242450.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2_df['submission_text'] = findSubmissionText(test2_df)
/tmp/ipykernel_13970/4182242450.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,index,text,label,id,subreddit,submission_id,submission_text,comment_num
0,13734,i concede the point!..well rather chuffed tha...,1,hznayrn,judaism,t89mft,"TIL, a philosemitist (sp?) is a non-Jewish per...",16
1,16286,they exist in the us too. its called {{organiz...,1,hyti8zy,judaism,t3gtzw,Israel’s Top Holocaust Institution Slams Putin...,15
2,22477,it irks me so much the way kaddish is fetishiz...,1,ijhy70m,judaism,wj7wjh,"Kaddish, Revised: How I mourned my dad a decad...",7
3,37725,right so in my analogy one could argue the ed...,1,icpuni8,judaism,ve6jvl,Does the recognition of 6 (or 8?) genders in t...,16
4,37896,you really think this conversation is equal to...,1,idt6555,judaism,vjgv13,Just curious - I know Muslims and some Christi...,18


# Creating Another TestDf for Immigration

In [10]:
immigration_df = df[df['subreddit'] == 'immigration']
immigration_df = immigration_df[immigration_df['label'] == 1]
immigration_df = immigration_df.reset_index(drop=False)

test3_df = immigration_df[:1000]
test3_df['submission_id'] = findSubmission(test3_df)
test3_df['submission_text'] = findSubmissionText(test3_df)
test3_df['comment_num'] = findSubmissionCommentNum(test3_df)

test3_df = test3_df[~test3_df['submission_text'].str.contains(r'\[removed\]')]
test3_df = test3_df[test3_df['comment_num'] < 20]
test3_df = test3_df.reset_index(drop=True)
test3_df.head()

/tmp/ipykernel_13970/2855155586.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test3_df['submission_id'] = findSubmission(test3_df)
/tmp/ipykernel_13970/2855155586.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test3_df['submission_text'] = findSubmissionText(test3_df)
/tmp/ipykernel_13970/2855155586.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,index,text,label,id,subreddit,submission_id,submission_text,comment_num
0,194,wait you said you have an approved tn status ...,1,ikm4zy4,immigration,wqd2sb,Entering the US Prior to the 10 Day requiremen...,7
1,8202,your parents have more skills then you so th...,1,i7k0wn9,immigration,ujj7b5,Immigration from Iran to a English Speaking co...,7
2,12535,"""bitcoin mining is not """"working"""" but more l...",1,i29i2jl,immigration,tp1d1r,Mine bitcoin - Can I mine bitcoin on f1(underg...,3
3,14623,i am a ukrainian and i have just become acquai...,1,i3lcbn7,immigration,txdjwi,From BTC usa to spain - Hello friends iwas det...,4
4,20929,""" \[url="""" """"\] 2022\[/url\] . . \[u...",1,i57t4ri,immigration,u657wt,United States border inadmissiblility - Incons...,8


# Using GPT-4 for Classification

Concern: It has to be the Parent Comment/Submission, not just in response to Submission

Concern: Rule takes up alot of tokens, maybe try making chatbot

#### promptGPT(df, rules)

In [11]:
import os
import openai
import creds
import time

openai.api_key = creds.openai_api_key

def promptGPT(df, rules, num):
    response = []
    total_responses = []
    amount = 300

    for index, row in df.iterrows():
        
        prompt = f"""
        Original Comment separated by the delimiter `: 
        
        `{row['submission_text']}`
        
        Reply Comment separated by the delimiter `: 
        
        `{row["text"]}`
        
        Here are the current rules of the community separated by the delimiter `: 
        
        `{rules}`
        
        Can you infer why Reply Comment was removed based on the guidelines? If Reply Comment does not align with any of the {num} rules, state so and what the reason for removal is.
        If it does align, use this format separated by delimeter ` as a response: `Rule X` where X is a number 1-{num}.
        If it does not align, say "None" then provide the reason for removal.
        """

        response = openai.ChatCompletion.create(
            model="gpt-4",  
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,  
            n = 1, 
            stop=None  #
        )
        time.sleep(1)
        
        total_responses.append(response)
        
    return total_responses

        

#### For r/islam

In [12]:
rules = """
    1. Be respectful at all times and conduct yourself in a civil manner. 
    2. No personal information or illegal content.
    3. No harassment or witch-hunting.
    4. Do not derail posts.
    5. Do not proselytize.
    6. Do not engage in behaviour that encourages vote manipulation or brigading.
    7. Do not post any NSFW content without prior approval by a moderator.
    8. Do not engage in sectarianism.
    9. Do not give or imply any rulings or religious edicts.
    10. No advertising, self-promotion, fundraising, or data collection.
    11. No FAQs or posts addressed in the wiki.
    12. All posts must meet the submission guidelines."""
    
gptResponses = promptGPT(test_df[:10], rules, 12)

In [13]:
analysisdf = test_df[:10]

gptList = []

for i in range(len(gptResponses)):
    gptList.append(gptResponses[i]['choices'][0]['message']['content'])
    
analysisdf['gpt_4_responses'] = gptList
analysisdf.head()

/tmp/ipykernel_13970/2859504507.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysisdf['gpt_4_responses'] = gptList


,index,text,label,id,subreddit,submission_id,submission_text,comment_num,gpt_4_responses
0,1980,if you missed them intentionally than you need...,1,iiejfel,islam,wctaej,Missed Zuhr and Ishaa prayers intentionally. -...,4,`Rule 9`
1,2184,and aisha was a virgin until the end of her life,1,i3uzwgw,islam,tyeg2p,So for some reason this islamaphobic argument ...,12,None. The comment was removed not because of a...
2,4094,it's not a satanic pentagram you uncultured sw...,1,i81xj5x,islam,ulzfbo,Moroccan Ministry of Awqaf and Islamic Affairs...,3,`Rule 1`
3,4826,haram haram. just read the quran and allah wil...,1,i194hqb,islam,thnice,Is Medical Assisted Suicide Halal? - Say you h...,4,`Rule 9`
4,4861,is a 100km journey by car difficult enough to ...,1,i4js4ad,islam,u2hg74,"Can I break my fast? - Hey, I am going to anot...",3,`Rule 1`


In [ ]:
analysisdf.to_csv('/home/dachacon/GitHub/reddit-req-engr/csv/islam_gpt_result.csv')

#### For r/judaism

In [21]:
rules = """"
1. Don't be a jerk
2. Don't proselytize.
3. No antisemitism.
4. Don't make clickbait.
5. Don't fundraise/survey without prior permission.
6. If it's worth reporting, report and don't respond
7. Reports are most useful when you give details
8. No Politics
9. No reposts
"""

gptResponses = promptGPT(test2_df[:10], rules, 9)

In [22]:
analysisdf = test2_df[:10]

gptList = []

for i in range(len(gptResponses)):
    gptList.append(gptResponses[i]['choices'][0]['message']['content'])
    
analysisdf['gpt_4_responses'] = gptList
analysisdf.head()

/tmp/ipykernel_13970/2318126990.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysisdf['gpt_4_responses'] = gptList


,index,text,label,id,subreddit,submission_id,submission_text,comment_num,gpt_4_responses
0,13734,i concede the point!..well rather chuffed tha...,1,hznayrn,judaism,t89mft,"TIL, a philosemitist (sp?) is a non-Jewish per...",16,"""None"" The reply comment was likely removed be..."
1,16286,they exist in the us too. its called {{organiz...,1,hyti8zy,judaism,t3gtzw,Israel’s Top Holocaust Institution Slams Putin...,15,`None` The reply comment was likely removed be...
2,22477,it irks me so much the way kaddish is fetishiz...,1,ijhy70m,judaism,wj7wjh,"Kaddish, Revised: How I mourned my dad a decad...",7,None. The comment may have been interpreted as...
3,37725,right so in my analogy one could argue the ed...,1,icpuni8,judaism,ve6jvl,Does the recognition of 6 (or 8?) genders in t...,16,None. The reply comment does not appear to vio...
4,37896,you really think this conversation is equal to...,1,idt6555,judaism,vjgv13,Just curious - I know Muslims and some Christi...,18,`Rule 1`


In [ ]:
analysisdf.to_csv('/home/dachacon/GitHub/reddit-req-engr/csv/judaism_gpt_result.csv')

#### For immigration

In [ ]:
rules = """"
1. Incivility, Personal Attacks, etc.
2. Hate-Speech, Xenophobia, Anti-Immigration, etc.
3. Blogspam/Advertising
4. Editorialized Headline
5. Off-Topic Post
6. Asking for Money
7. Asking How to Break the Law
8. Advising Obvious Illegal Activity
9. Surveys and Interview Requests
"""

gptResponses = promptGPT(test3_df[:10], rules, 9)

In [ ]:
analysisdf = test3_df[:10]

gptList = []

for i in range(len(gptResponses)):
    gptList.append(gptResponses[i]['choices'][0]['message']['content'])
    
analysisdf['gpt_4_responses'] = gptList
analysisdf.head()

In [ ]:
analysisdf.to_csv('/home/dachacon/GitHub/reddit-req-engr/csv/immigration_gpt_result.csv')

# Using Llama-2 for Classification

In [16]:
from transformers import pipeline, Conversation

chatbot = pipeline('conversational', model = "/data/shared/llama2/llama/7B-Chat/", device = 'cpu', max_length = 2000)

/data/installation/anaconda3/envs/reddit-req-eng/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.84s/it]


In [29]:
from itertools import islice

def promptLlama(df, rules, num):
    response = []
    total_responses = []
    amount = 300

    prompt = f"""
    Original Comment separated by the delimiter `: 
    
    `{df['submission_text'][0]}`
    
    Reply Comment separated by the delimiter `: 
    
    `{df["text"][0]}`
    
    Here are the current rules of the community separated by the delimiter `: 
    
    `{rules}`
    
    Can you infer why Reply Comment was removed based on the guidelines? If Reply Comment does not align with any of the {num} rules, state so and what the reason for removal is.
    If it does align, use this format separated by delimeter ` as a response: `Rule X` where X is a number 1-{num}.
    If it does not align, say "None" then provide the reason for removal.
    """
    
    conv = Conversation(prompt)
    conv = chatbot(conv)
    total_responses.append(conv.generated_responses[-1])
        
    for index, row in islice(df.iterrows(), 1, None):
        prompt = f"""
        Original Comment separated by the delimiter `: 
        
        `{row['submission_text']}`
        
        Reply Comment separated by the delimiter `: 
        
        `{row["text"]}`
        
        Can you infer why Reply Comment was removed based on the previous guidelines? If Reply Comment does not align with any of the {num} rules, state so and what the reason for removal is.
        If it does align, use this format separated by delimeter ` as a response: `Rule X` where X is a number 1-{num}.
        If it does not align, say "None" then provide the reason for removal.
        """
        
        conv = Conversation(prompt)
        conv = chatbot(conv)
        
        total_responses.append(conv.generated_responses[-1])

    return total_responses



#### For r/islam

In [18]:
rules = """
    1. Be respectful at all times and conduct yourself in a civil manner. 
    2. No personal information or illegal content.
    3. No harassment or witch-hunting.
    4. Do not derail posts.
    5. Do not proselytize.
    6. Do not engage in behaviour that encourages vote manipulation or brigading.
    7. Do not post any NSFW content without prior approval by a moderator.
    8. Do not engage in sectarianism.
    9. Do not give or imply any rulings or religious edicts.
    10. No advertising, self-promotion, fundraising, or data collection.
    11. No FAQs or posts addressed in the wiki.
    12. All posts must meet the submission guidelines."""
    
llamaResponses = promptLlama(test_df[:10],rules, 12)


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



In [19]:
llamaList = []

for i in range(len(llamaResponses)):
    llamaList.append(llamaResponses[i])
    
analysisdf['llama_responses'] = llamaList[:10]
analysisdf.head()

/tmp/ipykernel_13970/583306909.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysisdf['llama_responses'] = llamaList[:10]


,index,text,label,id,subreddit,submission_id,submission_text,comment_num,llama_responses
0,1980,if you missed them intentionally than you need...,1,iiejfel,islam,wctaej,Missed Zuhr and Ishaa prayers intentionally. -...,4,"Based on the guidelines provided, the Reply C..."
1,2184,and aisha was a virgin until the end of her life,1,i3uzwgw,islam,tyeg2p,So for some reason this islamaphobic argument ...,12,"Based on the provided guidelines, the Reply C..."
2,4094,it's not a satanic pentagram you uncultured sw...,1,i81xj5x,islam,ulzfbo,Moroccan Ministry of Awqaf and Islamic Affairs...,3,Based on the provided comment and reply comme...
3,4826,haram haram. just read the quran and allah wil...,1,i194hqb,islam,thnice,Is Medical Assisted Suicide Halal? - Say you h...,4,Based on the provided comment and reply comme...
4,4861,is a 100km journey by car difficult enough to ...,1,i4js4ad,islam,u2hg74,"Can I break my fast? - Hey, I am going to anot...",3,Based on the provided comment and reply comme...


In [20]:
analysisdf.to_csv('/home/dachacon/GitHub/reddit-req-engr/csv/islam_full_result.csv')

#### For r/Judaism

In [30]:
rules = """"
1. Don't be a jerk
2. Don't proselytize.
3. No antisemitism.
4. Don't make clickbait.
5. Don't fundraise/survey without prior permission.
6. If it's worth reporting, report and don't respond
7. Reports are most useful when you give details
8. No Politics
9. No reposts
"""

llamaResponses = promptLlama(test2_df[:10], rules, 9)

In [31]:
llamaList = []

for i in range(len(llamaResponses)):
    llamaList.append(llamaResponses[i])
    
analysisdf['llama_responses'] = llamaList[:10]
analysisdf.head()

/tmp/ipykernel_13970/2085993233.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysisdf['llama_responses'] = llamaList[:10]


,index,text,label,id,subreddit,submission_id,submission_text,comment_num,gpt_4_responses,llama_responses
0,13734,i concede the point!..well rather chuffed tha...,1,hznayrn,judaism,t89mft,"TIL, a philosemitist (sp?) is a non-Jewish per...",16,"""None"" The reply comment was likely removed be...","Based on the provided guidelines, I can infer..."
1,16286,they exist in the us too. its called {{organiz...,1,hyti8zy,judaism,t3gtzw,Israel’s Top Holocaust Institution Slams Putin...,15,`None` The reply comment was likely removed be...,"Based on the previous guidelines, the Reply C..."
2,22477,it irks me so much the way kaddish is fetishiz...,1,ijhy70m,judaism,wj7wjh,"Kaddish, Revised: How I mourned my dad a decad...",7,None. The comment may have been interpreted as...,"Based on the provided guidelines, the Reply C..."
3,37725,right so in my analogy one could argue the ed...,1,icpuni8,judaism,ve6jvl,Does the recognition of 6 (or 8?) genders in t...,16,None. The reply comment does not appear to vio...,Based on the provided comment and reply comme...
4,37896,you really think this conversation is equal to...,1,idt6555,judaism,vjgv13,Just curious - I know Muslims and some Christi...,18,`Rule 1`,"Based on the provided comments, here is why t..."


In [32]:
analysisdf.to_csv('/home/dachacon/GitHub/reddit-req-engr/csv/judaism_full_result.csv')